In [ ]:
import boto3

session=boto3.session.Session(profile_name="ais-admin-gmail-profile")
ecs=session.client('ecs')
efs=session.client('efs')
ec2=session.client('ec2')
iam=session.client('iam')
ec2_res=session.resource('ec2')


In [93]:
import boto3 
import json
import time

class AIS_INFRA:
    def __init__(self,session):
        self.session=session
        self.objects={}
    def check_status(self):
        ec2=self.session.client('ec2')
        efs=self.session.client('efs')
        ec2_res=self.session.resource('ec2')
        ecs=self.session.client('ecs')
        self.objects["AWS_VPC_ID"] = [vpc["VpcId"] for vpc in ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': ['ais_vpc'] }])["Vpcs"]]
        self.objects["AWS_SUBNET_ID"]=[ s.id for s in ec2_res.subnets.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_subnet'] }])]
        self.objects["AWS_INTERNET_GATEWAY_ID"]=[ g.id for g in ec2_res.internet_gateways.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_gateway'] }])]
        self.objects["AWS_ROUTE_TABLE_ID"]=[ rt.id for rt in ec2_res.route_tables.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_route_table'] }])]
        self.objects["AWS_EC2_SECURITY_GROUP_ID"]=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_ec2_security_group'] }])]
        self.objects["AWS_EFS_SECURITY_GROUP_ID"]=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_efs_security_group'] }])]
        self.objects["AWS_WEBAPP_SECURITY_GROUP_ID"]=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_webapp_security_group'] }])]
        self.objects["AWS_STREAMING_SECURITY_GROUP_ID"]=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_streaming_security_group'] }])]
        self.objects["AWS_REDIS_SECURITY_GROUP_ID"]=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_redis_security_group'] }])]
        self.objects["AWS_DEFAULT_SECURITY_GROUP_ID"]=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_default_security_group'] }])]
        self.objects["AWS_EFS_ID"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"]
        self.objects["AWS_AIS_CLUSTER_ARN"]=[c["clusterArn"] for c in ecs.describe_clusters(clusters=["ais-cluster"])["clusters"]]
    def get_objects(self):
        return self.objects
    def create_vpc_objects(self):
        ## Create a VPC with DNS hostname enabled
        ec2_res=self.session.resource('ec2')
        ec2Client = self.session.client('ec2')
        vpc = ec2_res.create_vpc(CidrBlock='10.0.0.0/16')
        vpc.create_tags(Tags=[{"Key": "Name", "Value": "ais_vpc"}])
        vpc.wait_until_available()
        ec2Client.modify_vpc_attribute( VpcId = vpc.id , EnableDnsSupport = { 'Value': True } )
        ec2Client.modify_vpc_attribute( VpcId = vpc.id , EnableDnsHostnames = { 'Value': True } )
        print("VPC created with id:{}".format(vpc.vpc_id))
        ## Create a public subnet
        subnet=ec2_res.create_subnet(VpcId=vpc.id, CidrBlock='10.0.1.0/24',AvailabilityZone="eu-west-2a")
        ec2Client.modify_subnet_attribute(SubnetId=subnet.id,MapPublicIpOnLaunch={'Value': True})
        subnet.create_tags(Tags=[{"Key": "Name", "Value": "ais_subnet"}])
        print("Subnet created with Id:{}".format(subnet.id))
        ## Create an Internet Gateway and attach to the VPC
        gw=ec2_res.create_internet_gateway()
        vpc.attach_internet_gateway(InternetGatewayId = gw.id)
        gw.create_tags(Tags=[{"Key": "Name", "Value": "ais_gateway"}])
        print("Internet Gateway created with Id:{}".format(gw.id))
        ## Create a custom route table with route to internet gateway and explicite assosiation to public subnet
        rt = vpc.create_route_table()
        route = rt.create_route(DestinationCidrBlock='0.0.0.0/0',GatewayId=gw.id)
        rt.associate_with_subnet(SubnetId=subnet.id)
        rt.create_tags(Tags=[{"Key": "Name", "Value": "ais_route_table"}])
        print("Route Table created with Id:{}".format(rt.id))
        ## Create a custom security group with ingress rules for EC2
        sec_group_ec2 = ec2_res.create_security_group(GroupName='ais-ec2-security-group', Description='AIS EC2 security group', VpcId=vpc.id)
        sec_group_ec2.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 22, "ToPort": 22, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow SSH"}]}])
        sec_group_ec2.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 80, "ToPort": 80, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow HTTP"}]}])
        sec_group_ec2.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 443, "ToPort": 443, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow HTTPS"}]}])
        sec_group_ec2.create_tags(Tags=[{"Key": "Name", "Value": "ais_ec2_security_group"}])
        print("EC2 Security Group created with Id:{}".format(sec_group_ec2.id))
        ## Create a custom security group with ingress rules for EFS
        sec_group_efs = ec2_res.create_security_group(GroupName='ais-efs-security-group', Description='AIS EFS security group', VpcId=vpc.id)
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 2049, "ToPort": 2049, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow NFS"}]}])
        sec_group_efs.create_tags(Tags=[{"Key": "Name", "Value": "ais_efs_security_group"}])
        print("EFS Security Group created with Id:{}".format(sec_group_efs.id))
        ## Create a custom security group with ingress rules for Fargate tasks
        sec_group_efs = ec2_res.create_security_group(GroupName='ais-streaming-security-group', Description='AIS EFS security group for streaming tasks', VpcId=vpc.id)
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "-1", "FromPort": -1 , "ToPort": -1, "IpRanges": [{"CidrIp": "10.0.1.0/24","Description": "Allow All traffic from localnet"}]}])
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "udp", "FromPort": 1234, "ToPort": 1244, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow UDP streams"}]}])
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 1234, "ToPort": 1244, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow TCP streams"}]}])
        sec_group_efs.create_tags(Tags=[{"Key": "Name", "Value": "ais_streaming_security_group"}])
        print("Streaming Security Group created with Id:{}".format(sec_group_efs.id))
        sec_group_efs = ec2_res.create_security_group(GroupName='ais-webapp-security-group', Description='AIS Webapp security group for streaming tasks', VpcId=vpc.id)
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "-1", "FromPort": -1, "ToPort": -1 , "IpRanges": [{"CidrIp": "10.0.1.0/24","Description": "Allow All traffic from localnet"}]}])
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 80, "ToPort": 80, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow HTTP port"}]}])
        sec_group_efs.create_tags(Tags=[{"Key": "Name", "Value": "ais_webapp_security_group"}])
        print("Webapp Security Group created with Id:{}".format(sec_group_efs.id))
        sec_group_efs = ec2_res.create_security_group(GroupName='ais-redis-security-group', Description='AIS Redis security group for Redis task', VpcId=vpc.id)
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "-1", "FromPort": -1, "ToPort": -1 , "IpRanges": [{"CidrIp": "10.0.1.0/24","Description": "Allow All traffic from localnet"}]}])
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "tcp", "FromPort": 6379, "ToPort": 6379, "IpRanges": [{"CidrIp": "0.0.0.0/0","Description": "Allow Redis port"}]}])
        sec_group_efs.create_tags(Tags=[{"Key": "Name", "Value": "ais_redis_security_group"}])
        print("Redis Security Group created with Id:{}".format(sec_group_efs.id))
        sec_group_efs = ec2_res.create_security_group(GroupName='ais-default-security-group', Description='AIS default security group for streaming tasks', VpcId=vpc.id)
        sec_group_efs.authorize_ingress(IpPermissions=[{"IpProtocol": "-1", "FromPort": -1, "ToPort": -1, "IpRanges": [{"CidrIp": "10.0.1.0/24","Description": "Allow All traffic from localnet"}]}])
        sec_group_efs.create_tags(Tags=[{"Key": "Name", "Value": "ais_default_security_group"}])
        print("Default Security Group created with Id:{}".format(sec_group_efs.id))

    def delete_vpc_objects(self):
        ec2_res=self.session.resource('ec2')
        for vpcid in self.objects["AWS_VPC_ID"]:
            for sg_id in self.objects["AWS_EC2_SECURITY_GROUP_ID"]:
                ec2_res.SecurityGroup(sg_id).delete()
            for sg_id in self.objects["AWS_EFS_SECURITY_GROUP_ID"]:
                ec2_res.SecurityGroup(sg_id).delete()
            for sg_id in self.objects["AWS_WEBAPP_SECURITY_GROUP_ID"]:
                ec2_res.SecurityGroup(sg_id).delete()
            for sg_id in self.objects["AWS_STREAMING_SECURITY_GROUP_ID"]:
                ec2_res.SecurityGroup(sg_id).delete()
            for sg_id in self.objects["AWS_REDIS_SECURITY_GROUP_ID"]:
                ec2_res.SecurityGroup(sg_id).delete()
            for sg_id in self.objects["AWS_DEFAULT_SECURITY_GROUP_ID"]:
                ec2_res.SecurityGroup(sg_id).delete()
            for rt_id in self.objects["AWS_ROUTE_TABLE_ID"]:
                for ra in ec2_res.RouteTable(rt_id).associations:
                    ra.delete()
                for r in ec2_res.RouteTable(rt_id).routes:
                    if r.origin=="CreateRoute":
                        r.delete()
                ec2_res.RouteTable(rt_id).delete()
            for gid in self.objects["AWS_INTERNET_GATEWAY_ID"]:
                for a in ec2_res.InternetGateway(gid).attachments:
                    ec2_res.InternetGateway(gid).detach_from_vpc(VpcId=a["VpcId"])
                ec2_res.InternetGateway(gid).delete()
            for sid in self.objects["AWS_SUBNET_ID"]:
                ec2_res.Subnet(sid).delete()
            ec2_res.Vpc(vpcid).delete()
            print("Deleted VPC objects.")
            
    def create_iam_roles(self):
        ## Create an IAM EC2 service role and attach AmazonEC2ContainerServiceforEC2Role policy
        arpd=json.dumps({ "Version": "2008-10-17",
                          "Statement": [
                            {
                              "Effect": "Allow",
                              "Principal": {
                                "Service": "ec2.amazonaws.com"
                              },
                              "Action": "sts:AssumeRole"
                            }
                          ]
                        })
        client = self.session.client('iam')
        role = client.create_role(RoleName="ecsInstanceRole",AssumeRolePolicyDocument=arpd)
        response = client.attach_role_policy(RoleName='ecsInstanceRole',PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2ContainerServiceforEC2Role')
        ## Create an IAM Instance Profile for Your ECS EC2 Instances using ecsInstanceRole role
        response = client.create_instance_profile(InstanceProfileName='ecsInstanceProfileRole')
        client.add_role_to_instance_profile(InstanceProfileName='ecsInstanceProfileRole',RoleName='ecsInstanceRole')
        print("IAM roles for AIS created. ecsInstanceProfileRole ARN:{}".format(response["InstanceProfile"]["Arn"]))
                                                                      
    def delete_iam_roles(self):
        client = self.session.client('iam')
        response = client.remove_role_from_instance_profile(InstanceProfileName='ecsInstanceProfileRole',RoleName='ecsInstanceRole')
        response = client.delete_instance_profile(InstanceProfileName='ecsInstanceProfileRole')
        response = client.detach_role_policy(RoleName='ecsInstanceRole', PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2ContainerServiceforEC2Role')
        response = client.delete_role(RoleName='ecsInstanceRole')
        print("Deleted IAM roles.")
        
    def create_efs_objects(self):
        client = self.session.client('efs')
        efs={}
        if len([fs for fs in client.describe_file_systems(CreationToken="ais_efs")["FileSystems"]])==0:
            efs = client.create_file_system(CreationToken='ais_efs', PerformanceMode='generalPurpose',
                    Tags=[{'Key': 'Name','Value': 'ais_efs'}])
            print("EFS created with Id:{}".format(efs["FileSystemId"]))
            for i in range(1,60):
                if client.describe_file_systems(FileSystemId=efs["FileSystemId"])["FileSystems"][0]["LifeCycleState"]=="available":
                    break
                time.sleep(1)
        else:
            print("EFS with token ais_efs already exists.")
            efs=fs
        ec2_res=self.session.resource('ec2')
        subnet=[ s.id for s in ec2_res.subnets.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_subnet'] }])][0]
        sec_group=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_efs_security_group'] }])]
        resp=client.create_mount_target(FileSystemId=efs["FileSystemId"],SubnetId=subnet,SecurityGroups=sec_group)
        print("EFS mount target created with Id:{} IP:{}".format(resp["MountTargetId"],resp["IpAddress"]))
        
    def delete_efs_objects(self):
        efs = self.session.client('efs')
        for fsid in [fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"]:
            for mt in efs.describe_mount_targets(FileSystemId=fsid)["MountTargets"]:
                efs.delete_mount_target(MountTargetId=mt["MountTargetId"])
            while len(efs.describe_mount_targets(FileSystemId=fsid)["MountTargets"])>0:
                time.sleep(3)
            efs.delete_file_system(FileSystemId=fsid)
        print("Deleted EFS objects.")
    
    def create_ecs_objects(self):
        ecs=self.session.client('ecs')
        # Create ECS cluster with name ais-cluster
        cluster = ecs.create_cluster(clusterName='ais-cluster')
        print("ais-cluster created with ARN:{}".format(cluster["cluster"]["clusterArn"]))
        
    def delete_ecs_objects(self):
        ecs=self.session.client('ecs')
        ecs.delete_cluster(cluster="ais-cluster")
        print("ais-cluster deleted.")
        
    def create_task_definitions(self):
        ecs=self.session.client('ecs')
        efs = self.session.client('efs')
        log = self.session.client("logs")
        
        with open('ais_webapp.json') as ais_webapp:
            ab=json.load(ais_webapp)
            ab["volumes"][0]["efsVolumeConfiguration"]["fileSystemId"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"][0]
            res=ecs.register_task_definition(**ab)
            print("ais_webapp task definition registered with ARN:{}".format(res["taskDefinition"]["taskDefinitionArn"]))
        if len(log.describe_log_groups(logGroupNamePrefix="ais_webapp")["logGroups"])==0:
            log.create_log_group(logGroupName="ais_webapp")
        with open('ais_redis.json') as ais_redis:
            ab=json.load(ais_redis)
            ab["volumes"][0]["efsVolumeConfiguration"]["fileSystemId"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"][0]
            res=ecs.register_task_definition(**ab)
            print("ais_redis task definition registered with ARN:{}".format(res["taskDefinition"]["taskDefinitionArn"]))
        if len(log.describe_log_groups(logGroupNamePrefix="ais_redis")["logGroups"])==0:
            log.create_log_group(logGroupName="ais_redis")
        with open('ais_stream_nick.json') as ais_stream_nick:
            ab=json.load(ais_stream_nick)
            ab["volumes"][0]["efsVolumeConfiguration"]["fileSystemId"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"][0]
            res=ecs.register_task_definition(**ab)
            print("ais_stream_nick task definition registered with ARN:{}".format(res["taskDefinition"]["taskDefinitionArn"]))
        if len(log.describe_log_groups(logGroupNamePrefix="ais_stream_nick")["logGroups"])==0:
            log.create_log_group(logGroupName="ais_stream_nick")
        with open('ais_recorder.json') as ais_recorder:
            ab=json.load(ais_recorder)
            ab["volumes"][0]["efsVolumeConfiguration"]["fileSystemId"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"][0]
            res=ecs.register_task_definition(**ab)
            print("ais_recorder task definition registered with ARN:{}".format(res["taskDefinition"]["taskDefinitionArn"]))
        if len(log.describe_log_groups(logGroupNamePrefix="ais_recorder")["logGroups"])==0:
            log.create_log_group(logGroupName="ais_recorder")
        with open('ais_encoder.json') as ais_encoder:
            ab=json.load(ais_encoder)
            ab["volumes"][0]["efsVolumeConfiguration"]["fileSystemId"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"][0]
            res=ecs.register_task_definition(**ab)
            print("ais_encoder task definition registered with ARN:{}".format(res["taskDefinition"]["taskDefinitionArn"]))
        if len(log.describe_log_groups(logGroupNamePrefix="ais_encoderr")["logGroups"])==0:
            log.create_log_group(logGroupName="ais_encoder")

    def delete_task_definitions(self):
        ecs=self.session.client('ecs')
        for ab_arn in ecs.list_task_definitions(familyPrefix='ais_webapp')["taskDefinitionArns"]:
            ecs.deregister_task_definition(taskDefinition=ab_arn)
        for ab_arn in ecs.list_task_definitions(familyPrefix='ais_redis')["taskDefinitionArns"]:
            ecs.deregister_task_definition(taskDefinition=ab_arn)
        for ab_arn in ecs.list_task_definitions(familyPrefix='ais_stream_nick')["taskDefinitionArns"]:
            ecs.deregister_task_definition(taskDefinition=ab_arn)
        for ab_arn in ecs.list_task_definitions(familyPrefix='ais_recorder')["taskDefinitionArns"]:
            ecs.deregister_task_definition(taskDefinition=ab_arn)
        for ab_arn in ecs.list_task_definitions(familyPrefix='ais_encoder')["taskDefinitionArns"]:
            ecs.deregister_task_definition(taskDefinition=ab_arn)
        print("Deleted task definitions.")
            
    def create_ec2_task_definitions(self):
        ecs=self.session.client('ecs')       
        efs = self.session.client('efs')
        with open('ais_stream_nick_ec2.json') as test:
            ab=json.load(test)
            ab["volumes"][0]["efsVolumeConfiguration"]["fileSystemId"]=[fs["FileSystemId"] for fs in efs.describe_file_systems()["FileSystems"] if fs["Name"]=="ais_efs"][0]
            res=ecs.register_task_definition(**ab)
            print("test task definition registered with ARN:{}".format(res["taskDefinition"]["taskDefinitionArn"]))

    def create_ec2_objects(self):
        ssm=self.session.client("ssm")
        ec2_res=self.session.resource('ec2')
        ec2 = self.session.client('ec2')
        iam=self.session.client('iam')
        userdata="#!/bin/bash\necho ECS_CLUSTER=ais-cluster >> /etc/ecs/ecs.config"
        # Get the prefered AMI ID for ECS Optimized instance
        AWS_AMI_ID=ssm.get_parameters(Names=["/aws/service/ecs/optimized-ami/amazon-linux-2/recommended/image_id"])["Parameters"][0]["Value"]
        sg=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_ec2_security_group'] }])]
        subnet=[ s.id for s in ec2_res.subnets.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_subnet'] }])]
        response = ec2.run_instances(ImageId=AWS_AMI_ID,InstanceType='t2.micro',KeyName='lajos2',MaxCount=1,MinCount=1,
            SecurityGroupIds=sg,SubnetId=subnet[0],UserData=userdata,IamInstanceProfile={'Name':'ecsInstanceProfileRole'})
        ec2.create_tags(Resources=[response["Instances"][0]["InstanceId"]],Tags=[{'Key':'Name','Value':'ais_ec2_instance'}])
        print("EC2 instance created with InstanceId:{}".format(response["Instances"][0]["InstanceId"]))
    def delete_ec2_objects(self):
        pass
    def create_services(self):
        ecs=self.session.client('ecs')
        ec2_res=self.session.resource('ec2')
        ec2=self.session.client('ec2')
        discovery=self.session.client("servicediscovery")
        subnets=[ s.id for s in ec2_res.subnets.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_subnet'] }])]
        streaming_secgroups=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_streaming_security_group'] }])]
        webapp_secgroups=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_webapp_security_group'] }])]
        redis_secgroups=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_redis_security_group'] }])]
        default_secgroups=[ sg.id for sg in ec2_res.security_groups.filter(Filters=[{'Name': 'tag:Name', 'Values': ['ais_default_security_group'] }])]
        namespace_id=""
        if len([n["Id"] for n in discovery.list_namespaces()["Namespaces"] if n["Name"]=="local"])>0:
            namespace_id=[n["Id"] for n in discovery.list_namespaces()["Namespaces"] if n["Name"]=="local"][0]
        else:
            op_id=discovery.create_private_dns_namespace(Name="local",Description="ais private dns",
                Vpc=[vpc["VpcId"] for vpc in ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': ['ais_vpc'] }])["Vpcs"]][0])["OperationId"]
            while(1):
                resp=discovery.get_operation(OperationId=op_id)
                if resp["Operation"]["Status"]=="SUCCESS":
                    namespace_id=resp["Operation"]["Targets"]["NAMESPACE"]
                    break
                time.sleep(1)
        print("Private DNS namespace:{}".format(namespace_id))
        services=discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":[namespace_id]}])["Services"]
        #####################  Create ais_stream_nick   ################
        if len([s for s in ecs.describe_services(cluster="ais-cluster",services=["ais_stream_nick"])["services"] if s["status"]=="ACTIVE"])==0:
            if len([s["Arn"] for s in services if s["Name"]=="ais_stream_nick"])==0:
                discovery.create_service(Name="ais_stream_nick",NamespaceId=namespace_id,
                    DnsConfig={"NamespaceId":namespace_id,"DnsRecords":[{"Type":"A","TTL":60}]})
                time.sleep(2)
            services=discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":[namespace_id]}])["Services"]
            ret=ecs.create_service(cluster="ais-cluster",serviceName="ais_stream_nick",taskDefinition="ais_stream_nick",
                desiredCount=0,launchType="FARGATE",platformVersion="1.4.0",schedulingStrategy="REPLICA",
                networkConfiguration={"awsvpcConfiguration" : {"subnets":subnets,"securityGroups":streaming_secgroups,"assignPublicIp":"ENABLED"}},
                serviceRegistries=[{"registryArn":[s["Arn"] for s in services if s["Name"]=="ais_stream_nick"][0]}])
            print("ais_stream_nick service created with ARN:{}".format(ret["service"]["serviceArn"]))
        else:
            print("ais_stream_nick service already exists. Delete it before re-creating.")

        #####################  Create ais_webapp   ################
        if len([s for s in ecs.describe_services(cluster="ais-cluster",services=["ais_webapp"])["services"] if s["status"]=="ACTIVE"])==0:
            if len([s["Arn"] for s in services if s["Name"]=="ais_webapp"])==0:
                discovery.create_service(Name="ais_webapp",NamespaceId=namespace_id,
                    DnsConfig={"NamespaceId":namespace_id,"DnsRecords":[{"Type":"A","TTL":60}]})
                time.sleep(2)
            services=discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":[namespace_id]}])["Services"]
            ret=ecs.create_service(cluster="ais-cluster",serviceName="ais_webapp",taskDefinition="ais_webapp",
                desiredCount=0,launchType="FARGATE",platformVersion="1.4.0",schedulingStrategy="REPLICA",
                networkConfiguration={"awsvpcConfiguration" : {"subnets":subnets,"securityGroups":webapp_secgroups,"assignPublicIp":"ENABLED"}},
                serviceRegistries=[{"registryArn":[s["Arn"] for s in services if s["Name"]=="ais_webapp"][0]}])
            print("ais_webapp service created with ARN:{}".format(ret["service"]["serviceArn"]))
        else:
            print("ais_webapp service already exists. Delete it before re-creating.")

        #####################  Create ais_redis   ################
        if len([s for s in ecs.describe_services(cluster="ais-cluster",services=["ais_redis"])["services"] if s["status"]=="ACTIVE"])==0:
            if len([s["Arn"] for s in services if s["Name"]=="ais_redis"])==0:
                discovery.create_service(Name="ais_redis",NamespaceId=namespace_id,
                    DnsConfig={"NamespaceId":namespace_id,"DnsRecords":[{"Type":"A","TTL":60}]})
                time.sleep(2)
            services=discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":[namespace_id]}])["Services"]
            ret=ecs.create_service(cluster="ais-cluster",serviceName="ais_redis",taskDefinition="ais_redis",
                desiredCount=0,launchType="FARGATE",platformVersion="1.4.0",schedulingStrategy="REPLICA",
                networkConfiguration={"awsvpcConfiguration" : {"subnets":subnets,"securityGroups":redis_secgroups,"assignPublicIp":"ENABLED"}},
                serviceRegistries=[{"registryArn":[s["Arn"] for s in services if s["Name"]=="ais_redis"][0]}])
            print("ais_redis service created with ARN:{}".format(ret["service"]["serviceArn"]))
        else:
            print("ais_redis service already exists. Delete it before re-creating.")

        #####################  Create ais_recorder   ################
        if len([s for s in ecs.describe_services(cluster="ais-cluster",services=["ais_recorder"])["services"] if s["status"]=="ACTIVE"])==0:
            if len([s["Arn"] for s in services if s["Name"]=="ais_recorder"])==0:
                discovery.create_service(Name="ais_recorder",NamespaceId=namespace_id,
                    DnsConfig={"NamespaceId":namespace_id,"DnsRecords":[{"Type":"A","TTL":60}]})
                time.sleep(2)
            services=discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":[namespace_id]}])["Services"]
            ret=ecs.create_service(cluster="ais-cluster",serviceName="ais_recorder",taskDefinition="ais_recorder",
                desiredCount=0,launchType="FARGATE",platformVersion="1.4.0",schedulingStrategy="REPLICA",
                networkConfiguration={"awsvpcConfiguration" : {"subnets":subnets,"securityGroups":webapp_secgroups,"assignPublicIp":"ENABLED"}},
                serviceRegistries=[{"registryArn":[s["Arn"] for s in services if s["Name"]=="ais_recorder"][0]}])
            print("ais_recorder service created with ARN:{}".format(ret["service"]["serviceArn"]))
        else:
            print("ais_recorder service already exists. Delete it before re-creating.")

        #####################  Create ais_encoder   ################
        if len([s for s in ecs.describe_services(cluster="ais-cluster",services=["ais_encoder"])["services"] if s["status"]=="ACTIVE"])==0:
            if len([s["Arn"] for s in services if s["Name"]=="ais_encoder"])==0:
                discovery.create_service(Name="ais_encoder",NamespaceId=namespace_id,
                    DnsConfig={"NamespaceId":namespace_id,"DnsRecords":[{"Type":"A","TTL":60}]})
                time.sleep(2)
            services=discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":[namespace_id]}])["Services"]
            ret=ecs.create_service(cluster="ais-cluster",serviceName="ais_encoder",taskDefinition="ais_encoder",
                desiredCount=0,launchType="FARGATE",platformVersion="1.4.0",schedulingStrategy="REPLICA",
                networkConfiguration={"awsvpcConfiguration" : {"subnets":subnets,"securityGroups":default_secgroups,"assignPublicIp":"ENABLED"}},
                serviceRegistries=[{"registryArn":[s["Arn"] for s in services if s["Name"]=="ais_encoder"][0]}])
            print("ais_encoder service created with ARN:{}".format(ret["service"]["serviceArn"]))
        else:
            print("ais_encoder service already exists. Delete it before re-creating.")
            
    def delete_services(self):
        ecs=self.session.client('ecs')
        discovery=self.session.client("servicediscovery")
        services=["ais_redis","ais_webapp","ais_stream_nick","ais_recorder","ais_encoder"]
        namespace_id=[]
        for s in services:
            ecs.delete_service(cluster="ais-cluster",service=s,force=True)
        if len([n["Id"] for n in discovery.list_namespaces()["Namespaces"] if n["Name"]=="local"])>0:
            namespace_id=[n["Id"] for n in discovery.list_namespaces()["Namespaces"] if n["Name"]=="local"]
            for s in discovery.list_services(Filters=[{"Name":"NAMESPACE_ID","Values":namespace_id}])["Services"]:
                discovery.delete_service(Id=s["Id"])
        for n in namespace_id:
            discovery.delete_namespace(Id=n)
        print("Services:{} were deleted.".format(services))
        
    
    def start_redis(self):
        ecs=self.session.client('ecs')
        for s in ecs.describe_services(cluster="ais-cluster",services=["ais_redis"])["services"]:
            if s["desiredCount"]==0 and s["status"]=="ACTIVE":
                ecs.update_service(cluster="ais-cluster",service=s["serviceName"],desiredCount=1)
                print("{} has been started.".format(s["serviceName"]))

    def start_services(self):
        ecs=self.session.client('ecs')
        for s in ecs.describe_services(cluster="ais-cluster",services=["ais_webapp","ais_stream_nick","ais_recorder","ais_encoder"])["services"]:
            if s["desiredCount"]==0 and s["status"]=="ACTIVE":
                ecs.update_service(cluster="ais-cluster",service=s["serviceName"],desiredCount=1)
                print("{} has been started.".format(s["serviceName"]))
        
    def stop_redis(self):
        ecs=self.session.client('ecs')
        for s in ecs.describe_services(cluster="ais-cluster",services=["ais_redis"])["services"]:
            if s["desiredCount"]==1 and s["status"]=="ACTIVE":
                ecs.update_service(cluster="ais-cluster",service=s["serviceName"],desiredCount=0)
                print("{} has been stopped.".format(s["serviceName"]))

    def stop_services(self):
        ecs=self.session.client('ecs')
        for s in ecs.describe_services(cluster="ais-cluster",services=["ais_webapp","ais_stream_nick","ais_recorder","ais_encoder"])["services"]:
            if s["desiredCount"]==1 and s["status"]=="ACTIVE":
                ecs.update_service(cluster="ais-cluster",service=s["serviceName"],desiredCount=0)
                print("{} has been stopped.".format(s["serviceName"]))
            
    def create_infra(self):
        self.create_vpc_objects()
        self.create_iam_roles()
        self.create_efs_objects()
        self.create_ecs_objects()
        self.create_task_definitions()
        self.create_services()
        
    def delete_infra(self):
        self.stop_services()
        self.stop_redis()
        self.delete_services()
        self.delete_task_definitions()
        self.delete_ecs_objects()
        self.delete_efs_objects()
        self.delete_iam_roles()
        self.check_status()
        self.delete_vpc_objects()
       

In [94]:
import boto3

ais_infra=AIS_INFRA(session=boto3.session.Session(profile_name="ais-admin-gmail-profile"))

In [98]:
ais_infra.check_status()
print(ais_infra.get_objects())

{'AWS_STREAMING_SECURITY_GROUP_ID': ['sg-04d4907492b2ee767'], 'AWS_SUBNET_ID': ['subnet-0ae72794b625a0c8f'], 'AWS_INTERNET_GATEWAY_ID': ['igw-09b8e5df4fd5407fd'], 'AWS_DEFAULT_SECURITY_GROUP_ID': ['sg-01e34da8b5008cf9f'], 'AWS_AIS_CLUSTER_ARN': [u'arn:aws:ecs:eu-west-2:862791113833:cluster/ais-cluster'], 'AWS_EFS_ID': [], 'AWS_WEBAPP_SECURITY_GROUP_ID': ['sg-01831941d13849648'], 'AWS_EFS_SECURITY_GROUP_ID': ['sg-0485ef18b0e7ff3f3'], 'AWS_EC2_SECURITY_GROUP_ID': ['sg-0a059719c112b4a71'], 'AWS_REDIS_SECURITY_GROUP_ID': ['sg-061b4969ef06d286f'], 'AWS_ROUTE_TABLE_ID': ['rtb-0cf32211739bb86c6'], 'AWS_VPC_ID': ['vpc-0bb1e3a86ca89c965']}


In [95]:
ais_infra.delete_infra()

Services:['ais_redis', 'ais_webapp', 'ais_stream_nick', 'ais_recorder', 'ais_encoder'] were deleted.
Deleted task definitions.
ais-cluster deleted.
Deleted EFS objects.


NoSuchEntityException: An error occurred (NoSuchEntity) when calling the RemoveRoleFromInstanceProfile operation: The role with name ecsInstanceRole cannot be found.

In [99]:
ais_infra.delete_vpc_objects()

Deleted VPC objects.
